In [ ]:
from dash import Dash, html, dcc, Output, Input, State
from datetime import datetime
import pandas as pd
import yfinance as yf

In [ ]:
filename = "nasdaq_screener.csv"
data = pd.read_csv(filename)

In [ ]:
data.head()

In [ ]:
data_options = []
for element in data.index:
    data_options.append({"label": data["Name"][element], "value": data["Symbol"][element]})

In [ ]:
heading = html.H1("Stock Ticker Web App")
stock_picker = html.Div([html.H2('Select a stock:'),
                        dcc.Dropdown(
                            id="dropdown",
                            options=data_options,
                            value = ["GOOG"],
                            multi=True
                        )])
date_picker = html.Div([html.H2("Select Data"),
                        dcc.DatePickerRange(
                            id="datepicker",
                            min_date_allowed = datetime(2015, 1, 1),
                            max_date_allowed=datetime(2023, 2, 1),
                            start_date=datetime(2023, 1, 1),
                            end_date=datetime(2023, 2, 1)
                        )])
button = html.Div([html.Button(
                        id="submit-button",
                        n_clicks=0,
                        children="Submit"
                    )])
graph = dcc.Graph(id="stock-graph")

In [ ]:
app = Dash()
app.layout = html.Div([heading, stock_picker, date_picker, button, graph])

In [ ]:
@app.callback(Output("stock-graph", "figure"),
              [Input("submit-button", "n_clicks")],
              [State("dropdown", "value"), 
              State("datepicker", "start_date"),
              State("datepicker", "end_date")])
def update_graph(number_of_clicks, stocks, start_date, end_date):
    start = datetime.strptime(start_date[:10], "%Y-%m-%d")
    end = datetime.strptime(end_date[:10], "%Y-%m-%d")
    graph_data = []
    for stock in stocks:
        new_df = yf.download(tickers=[stock], start=start, end=end)
        new_df.head()
        dates = []
        for row in range(len(new_df)):
            new_date = str(new_df.index[row])
            new_date = new_date[0:10]
            dates.append(new_date)
        
        new_df["Date"] = dates
        graph_data.append({
            "x": new_df["Date"],
            "y": new_df["Adj Close"],
            "name": stock
        })
    
    figure = {
        "data": graph_data,
        "layout": {
            "title": "Stock Data"
        }
    }

    return figure


In [ ]:
app.run_server(port = 5050)